In [1]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import pandas as pd

# Set up data paths
DATA_DIR = Path("data")
INPUT_DIR = DATA_DIR / "Attachment 1"
OUTPUT_DIR = Path("results/Q1_results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Available images: {len(list(INPUT_DIR.glob('*.jpg')))}")


Input directory: data/Attachment 1
Output directory: results/Q1_results
Available images: 0


In [2]:
# Core Image Processing Functions

def read_image(image_path):
    """Read and return an image from the specified path."""
    img = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Could not read image from {image_path}")
    return img

def gaussian_blur(image, kernel_size=(5, 5), sigma=1.5):
    """Apply Gaussian blur to reduce noise."""
    return cv2.GaussianBlur(image, kernel_size, sigma)

def convert_to_gray(image):
    """Convert image to grayscale."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def create_binary_threshold(gray_image):
    """Create binary threshold using Triangle method."""
    ret, binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_TRIANGLE)
    print(f"Threshold value: {ret}")
    return binary

def morphological_opening(binary_image, kernel_size=(5, 5), iterations=3):
    """Apply morphological opening to remove noise."""
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=iterations)


In [3]:
# Apple Detection Functions

def detect_red_objects(image):
    """Detect red objects (apples) in the image using color thresholding."""
    # Define red color range in BGR
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])
    
    # Create mask for red objects
    mask = cv2.inRange(image, lower_red, upper_red)
    
    # Apply mask to extract red objects
    red_objects = cv2.bitwise_and(image, image, mask=mask)
    
    return red_objects, mask

def enhance_red_saturation(image, red_factor=1.5):
    """Enhance red saturation to improve apple detection."""
    # Convert to HSV for better color manipulation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Create mask for red pixels
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])
    
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    
    # Enhance saturation for red pixels
    hsv[red_mask > 0, 1] = np.clip(hsv[red_mask > 0, 1] * red_factor, 0, 255)
    
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


In [4]:
# Apple Counting and Location Functions

def detect_apple_contours(processed_image, min_area=100):
    """Detect apple contours and return count with coordinates."""
    # Convert to grayscale if needed
    if len(processed_image.shape) == 3:
        gray = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)
    else:
        gray = processed_image
    
    # Find contours
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    apple_info = []
    apple_count = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > min_area:  # Filter out small noise
            # Calculate center coordinates
            M = cv2.moments(cnt)
            if M['m00'] != 0:  # Avoid division by zero
                center_x = int(M['m10'] / M['m00'])
                center_y = int(M['m01'] / M['m00'])
                
                # Get bounding rectangle
                x, y, w, h = cv2.boundingRect(cnt)
                
                apple_info.append({
                    'id': apple_count + 1,
                    'center_x': center_x,
                    'center_y': center_y,
                    'area': area,
                    'bbox': (x, y, w, h)
                })
                apple_count += 1
    
    return apple_count, apple_info, contours

def draw_apple_detection(image, apple_info, contours):
    """Draw detection results on the image."""
    result_image = image.copy()
    
    for i, info in enumerate(apple_info):
        # Draw bounding rectangle
        x, y, w, h = info['bbox']
        cv2.rectangle(result_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw center point
        cv2.circle(result_image, (info['center_x'], info['center_y']), 3, (0, 0, 255), -1)
        
        # Add text label
        cv2.putText(result_image, f"Apple {info['id']}", 
                   (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    return result_image


In [5]:
# Main Processing Pipeline

def process_single_image(image_path, visualize=True):
    """Process a single image to detect and count apples."""
    # Read image
    original_image = read_image(image_path)
    print(f"Processing: {image_path.name}")
    
    # Step 1: Enhance red saturation
    enhanced_image = enhance_red_saturation(original_image)
    
    # Step 2: Apply Gaussian blur
    blurred_image = gaussian_blur(enhanced_image)
    
    # Step 3: Detect red objects
    red_objects, red_mask = detect_red_objects(blurred_image)
    
    # Step 4: Apply morphological operations
    opened_mask = morphological_opening(red_mask)
    
    # Step 5: Detect apple contours and count
    apple_count, apple_info, contours = detect_apple_contours(opened_mask)
    
    # Step 6: Draw detection results
    result_image = draw_apple_detection(original_image, apple_info, contours)
    
    if visualize:
        # Create visualization
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        axes[0, 0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title('Original Image')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))
        axes[0, 1].set_title('Enhanced Red Saturation')
        axes[0, 1].axis('off')
        
        axes[0, 2].imshow(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
        axes[0, 2].set_title('Gaussian Blur')
        axes[0, 2].axis('off')
        
        axes[1, 0].imshow(red_mask, cmap='gray')
        axes[1, 0].set_title('Red Object Mask')
        axes[1, 0].axis('off')
        
        axes[1, 1].imshow(opened_mask, cmap='gray')
        axes[1, 1].set_title('Morphological Opening')
        axes[1, 1].axis('off')
        
        axes[1, 2].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        axes[1, 2].set_title(f'Result: {apple_count} Apples')
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return {
        'image_name': image_path.name,
        'apple_count': apple_count,
        'apple_locations': apple_info,
        'result_image': result_image
    }


In [6]:
# Test on a Single Image

# Get the first available image for testing
test_images = list(INPUT_DIR.glob("*.jpg"))
if test_images:
    test_image_path = test_images[0]  # Use first image for testing
    print(f"Testing on: {test_image_path}")
    
    # Process the test image
    result = process_single_image(test_image_path, visualize=True)
    
    print(f"\nResults for {result['image_name']}:")
    print(f"Total apples detected: {result['apple_count']}")
    print("\nApple locations:")
    for apple in result['apple_locations']:
        print(f"  Apple {apple['id']}: Center({apple['center_x']}, {apple['center_y']}), Area: {apple['area']}")
else:
    print("No images found in the input directory!")


No images found in the input directory!


In [7]:
# Batch Processing Function

def process_all_images(max_images=10, save_results=True):
    """Process multiple images and collect results."""
    image_files = list(INPUT_DIR.glob("*.jpg"))[:max_images]  # Limit for testing
    
    if not image_files:
        print("No images found!")
        return None
    
    results = []
    
    print(f"Processing {len(image_files)} images...")
    
    for image_path in image_files:
        try:
            result = process_single_image(image_path, visualize=False)
            results.append(result)
            
            # Save result image if requested
            if save_results:
                output_path = OUTPUT_DIR / f"result_{result['image_name']}"
                cv2.imwrite(str(output_path), result['result_image'])
            
            print(f"✓ {result['image_name']}: {result['apple_count']} apples")
            
        except Exception as e:
            print(f"✗ Error processing {image_path.name}: {e}")
    
    return results

# Process a batch of images
batch_results = process_all_images(max_images=5, save_results=True)


No images found!


In [8]:
# Create Summary Statistics

if batch_results:
    # Create summary DataFrame
    summary_data = []
    for result in batch_results:
        summary_data.append({
            'Image': result['image_name'],
            'Apple_Count': result['apple_count'],
            'Total_Area': sum([apple['area'] for apple in result['apple_locations']]),
            'Avg_Apple_Area': np.mean([apple['area'] for apple in result['apple_locations']]) if result['apple_locations'] else 0
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Display summary
    print("=== APPLE DETECTION SUMMARY ===")
    print(summary_df.to_string(index=False))
    
    # Save summary to Excel
    summary_path = OUTPUT_DIR / "apple_detection_summary.xlsx"
    summary_df.to_excel(summary_path, index=False)
    print(f"\nSummary saved to: {summary_path}")
    
    # Display statistics
    print(f"\n=== STATISTICS ===")
    print(f"Total images processed: {len(batch_results)}")
    print(f"Total apples detected: {summary_df['Apple_Count'].sum()}")
    print(f"Average apples per image: {summary_df['Apple_Count'].mean():.2f}")
    print(f"Max apples in single image: {summary_df['Apple_Count'].max()}")
    print(f"Min apples in single image: {summary_df['Apple_Count'].min()}")
else:
    print("No results to summarize.")


No results to summarize.
